In [1]:
import urllib.request
import numpy as np
import pandas as pd
import re

#### 1.分红数据

In [3]:
url1 = "http://quotes.money.163.com/f10/fhpg_002270.html#01d05"
with urllib.request.urlopen(url1) as i:
    html = i.read()

In [5]:
data1 = pd.read_html(html)[3]    # html中第4个表才是分红数据
col = np.array(data1.columns.levels[0])    # 读人DataFrame中的数据是双层列名的
data1.columns = ['公告日期','分红年度','送股','转增','派息','股权登记日','除权除息日','红股上市日']    # 对DataFrame重命名
if col[2] == '分红方案（每10股）':    # 判断分红方案是否按10股派息
    data1['派息'] = data1['派息'] / 10
data1

,公告日期,分红年度,送股,转增,派息,股权登记日,除权除息日,红股上市日
0,2022-05-17,2021,--,--,0.140,2022-05-20,2022-05-23,--
1,2021-10-15,2021,--,--,0.160,2021-10-21,2021-10-22,--
2,2021-06-02,2020,--,--,0.027,2021-06-08,2021-06-09,--
3,2020-09-09,2020,--,--,0.050,2020-09-15,2020-09-16,--
4,2020-06-04,2019,--,--,0.042,2020-06-09,2020-06-10,--
5,2019-07-04,2018,--,--,0.043,2019-07-09,2019-07-10,--
6,2018-06-22,2017,0,5,0.120,2018-06-28,2018-06-29,2018-06-29
7,2016-06-18,2015,--,--,0.080,2016-06-23,2016-06-24,--
8,2015-03-13,2014,--,--,0.050,2015-03-19,2015-03-20,--
9,2014-04-25,2013,--,--,0.050,2014-04-30,2014-05-05,--


#### 2.业绩预告

In [2]:
url2 = 'http://quotes.money.163.com/f10/yjyg_002270.html#01c03'
with urllib.request.urlopen(url2) as i:
    html = i.read()

data2 = pd.read_html(html)[3]    # 网页中每个预告独立成一个表，历史预告可以用4，5，6等依次提取
data2.[[0,1]]

,0,1
0,报告日期,2021-12-31
1,预测类型,预升
2,预测摘要,预计2021年1-12月归属于上市公司股东的净利润为：416943903.45元，与上年同期...
3,预测内容,预计2021年1-12月归属于上市公司股东的净利润为：416943903.45元，与上年同期...


In [62]:
# 从业绩预告中提取预增比例
prediction = data2.iloc[2, 1]
print(prediction)

预计2021年1-12月归属于上市公司股东的净利润为：416943903.45元，与上年同期相比变动幅度：45.13%。


In [79]:
forecast = re.findall(r"\d+\.?\d+%", prediction)    # 匹配百分数，可以是不带小数的。返回List
growrh_rate_estimate = float(forecast[0][0:-1]) / 100    #将list中的第一个string提取并丢弃最后一个百分号，转换成浮点数。
print(growrh_rate_estimate)


0.45130000000000003


In [44]:
#re.findall(r'\b\d+(?:\.\d+)?%?(?!\S)', prediction)
print(re.findall(r"\d+\.\d+%", prediction))

['45.13%', '45.13%', '17.71%', '3.35%', '9.25%']
